# Experiments

In [95]:
import pandas as pd
import tensorflow as tf

from milp import codify_network
from teste import get_minimal_explanation

# For type annotations
import numpy as np

## Glass

In [96]:
dataset_name = 'glass'

training_data = pd.read_csv(f'datasets/{dataset_name}/train.csv')
testing_data = pd.read_csv(f'datasets/{dataset_name}/test.csv')

dataframe = pd.concat([training_data, testing_data])

keras_model = tf.keras.models.load_model(f'datasets/{dataset_name}/model_2layers_{dataset_name}.h5')

data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

In [97]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

TODO: Get random `i`

In [98]:
# i = 134 is also a nice value to study
i = 138
print('i =', i)
network_input = data[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification: np.int64 = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 138
1/1 [==============================] - 0s 30ms/step
Predictions: (ndarray[ndarray[<class 'numpy.float32'>]]) 0.0007575714
Network output: (<class 'numpy.int64'>) 1


In [99]:
mdl_aux = mp_model.clone()

minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, n_classes, 'fischetti', output_bounds)
minimal_explanation

[docplex.mp.LinearConstraint[input1](x_0,EQ,2.967691214515491),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-1.408120229258977),
 docplex.mp.LinearConstraint[input6](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input7](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

### Improving the Explanation

In [100]:
import docplex

In [101]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

In [102]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input1](x_0,EQ,2.967691214515491),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-1.408120229258977),
 docplex.mp.LinearConstraint[input6](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input7](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

In [103]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

In [104]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,2.967691214515491),
 docplex.mp.LinearConstraint[input GE](x_0,GE,2.967691214515491),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.408120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.790702170757714),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.24127975754059),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.24127975754059),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.6037614142464092),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [105]:
def log_and_improve_explanation(minimal_explanation: list, epsilon: float):
	for constraint in minimal_explanation:
		testing_model.solve()
		print('Initial constraint:' + '\t', constraint)

		variable = constraint.lhs
		while testing_model.solution is None:
			if constraint.sense == docplex.mp.constants.ComparisonType.LE:
				if constraint.rhs.constant <= variable.ub:
					constraint.rhs += epsilon
				else:
					break
			elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
				if constraint.rhs.constant >= variable.lb:
					constraint.rhs -= epsilon
				else:
					break
			else:
				raise Exception('Constraint sense was neither LE nor GE')

			testing_model.solve()

		# Undo last operation
		if constraint.sense == docplex.mp.constants.ComparisonType.LE:
			constraint.rhs -= epsilon
		elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
			constraint.rhs += epsilon

		print('Final constraint:' + '\t', constraint)
		print()

In [107]:
log_and_improve_explanation(linear_constraints, epsilon=0.01)

Initial constraint:	 input LE: x_0 <= 2.967691214515491
Final constraint:	 input LE: x_0 <= 5.127691214515445

Initial constraint:	 input GE: x_0 >= 2.967691214515491
Final constraint:	 input GE: x_0 >= 1.427691214515511

Initial constraint:	 input LE: x_3 <= -1.408120229258977
Final constraint:	 input LE: x_3 <= -1.408120229258977

Initial constraint:	 input GE: x_3 >= -1.408120229258977
Final constraint:	 input GE: x_3 >= -1.518120229258977

Initial constraint:	 input LE: x_5 <= -0.790702170757714
Final constraint:	 input LE: x_5 <= -0.790702170757714

Initial constraint:	 input GE: x_5 >= -0.790702170757714
Final constraint:	 input GE: x_5 >= -0.790702170757714

Initial constraint:	 input LE: x_6 <= 4.24127975754059
Final constraint:	 input LE: x_6 <= 4.24127975754059

Initial constraint:	 input GE: x_6 >= 4.24127975754059
Final constraint:	 input GE: x_6 >= 4.24127975754059

Initial constraint:	 input LE: x_7 <= -0.3615292659832898
Final constraint:	 input LE: x_7 <= -0.36152926598

In [ ]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.127691214515445),
 docplex.mp.LinearConstraint[input GE](x_0,GE,1.427691214515511),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.518120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.790702170757714),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.24127975754059),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.24127975754059),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.6037614142464092),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [ ]:
number_of_inputs = len(dataframe.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

In [ ]:
improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.127691214515445),
 docplex.mp.LinearConstraint[input GE](x_0,GE,1.427691214515511),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.518120229258977),
 docplex.mp.LinearConstraint[input](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input](x_8,EQ,-0.6037614142464092)]

### Pretty Printing

In [ ]:
def get_variable_index(variable: docplex.mp.dvar.Var) -> int:
	index = variable.name.split('_')[1]
	return int(index)

In [ ]:
def represent_constraint_with_feature_name(constraint: docplex.mp.constr.LinearConstraint):
	variable = constraint.lhs
	index = get_variable_index(variable)
	feature_name = dataframe.columns[index]
	return f'{feature_name} {constraint.sense.operator_symbol} {constraint.rhs}'

In [ ]:
def simple_print_explanation(explanation: list[docplex.mp.constr.LinearConstraint]):
	for e in explanation:
		print(represent_constraint_with_feature_name(e))

In [ ]:
def group_by_name(constraints: list[docplex.mp.constr.LinearConstraint]):
	group = {}
	for c in constraints:
		variable = c.lhs
		i = get_variable_index(variable)
		feature_name = dataframe.columns[i]
		if feature_name not in group:
			group[feature_name] = []
		group[feature_name].append(c)

	return group

In [ ]:
def get_interval_distance(constraints: list[docplex.mp.constr.LinearConstraint]):
	if len(constraints) == 1:
		return 0
	elif len(constraints) == 2:
		c_0 = constraints[0]
		c_1 = constraints[1]
		assert c_0.lhs.name == c_1.lhs.name
		distance = c_0.rhs.constant - c_1.rhs.constant
		return abs(distance)
	else:
		ValueError()

In [ ]:
def pretty_print_explanation(explanation: list[docplex.mp.constr.LinearConstraint]):
	group = group_by_name(improved_explanation)
	for (feature, constraints) in group.items():
		if len(constraints) == 2:
			c_0 = constraints[0]
			c_1 = constraints[1]
			le_operator = docplex.mp.constants.ComparisonType.LE
			if c_0.sense == le_operator:
				c_plus = c_0
				c_minus = c_1
			else:
				c_minus = c_0
				c_plus = c_1
			print(f'{c_minus.rhs} {le_operator.operator_symbol} {feature} {le_operator.operator_symbol} {c_plus.rhs}')
		else:
			print(represent_constraint_with_feature_name(constraints[0]))

In [ ]:
pretty_print_explanation(improved_explanation)

1.427691214515511 <= RI <= 5.127691214515445
-1.518120229258977 <= Al <= -1.408120229258977
K == -0.790702170757714
Ca == 4.24127975754059
Ba == -0.3615292659832898
Fe == -0.6037614142464092


In [ ]:
group = group_by_name(improved_explanation)
for feature in group:
	print(get_interval_distance(group[feature]))

3.699999999999934
0.1100000000000001
0
0
0
0


### Anchor

In [ ]:
from anchor import utils

In [ ]:
d = utils.load_csv_dataset(
	data=f'datasets/{dataset_name}/test.csv',
	target_idx=-1,
	feature_names=['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','target'],
	skip_first=True
)

In [ ]:
from anchor import anchor_tabular

In [ ]:
explainer = anchor_tabular.AnchorTabularExplainer(
	d.class_names,
	d.feature_names,
	d.train,
	d.categorical_names)

In [ ]:
predict_fn = lambda x: tf.argmax(keras_model.predict(x)[0]).numpy().reshape(1)

In [ ]:
for a in d.train:
	a == data[i]

/tmp/ipykernel_19251/1890216175.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  a == data[i]


In [ ]:
exp = explainer.explain_instance(data[i, :-1], predict_fn)

1/1 [==============================] - 0s 28ms/step


In [ ]:
for name in exp.names():
	print(name)

RI > -0.64
K > -0.60
Na <= 0.84
Si > -0.81
Al > -0.57


In [ ]:
pretty_print_explanation(improved_explanation)

1.427691214515511 <= RI <= 5.127691214515445
-1.518120229258977 <= Al <= -1.408120229258977
K == -0.790702170757714
Ca == 4.24127975754059
Ba == -0.3615292659832898
Fe == -0.6037614142464092


In [ ]:
simple_print_explanation(improved_explanation)

RI <= 5.127691214515445
RI >= 1.427691214515511
Al <= -1.408120229258977
Al >= -1.518120229258977
K == -0.790702170757714
Ca == 4.24127975754059
Ba == -0.3615292659832898
Fe == -0.6037614142464092


In [ ]:
print('Precision: %f' % exp.precision())
print('Coverage:  %f' % exp.coverage())

Precision: 0.252684
Coverage:  0.386700


## Australian

## Glass

In [109]:
dataset_name = 'australian'

training_data = pd.read_csv(f'datasets/{dataset_name}/train.csv')
testing_data = pd.read_csv(f'datasets/{dataset_name}/test.csv')

dataframe = pd.concat([training_data, testing_data])

keras_model = tf.keras.models.load_model(f'datasets/{dataset_name}/model_4layers_{dataset_name}.h5')

data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

In [110]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

TODO: Get random `i`

In [113]:
# i = 134 is also a nice value to study
i = 138
print('i =', i)
network_input = data[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification: np.int64 = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 138
1/1 [==============================] - 0s 30ms/step
Predictions: (ndarray[ndarray[<class 'numpy.float32'>]]) 0.061345257
Network output: (<class 'numpy.int64'>) 1


In [114]:
mdl_aux = mp_model.clone()

minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, n_classes, 'fischetti', output_bounds)
minimal_explanation

### Improving the Explanation

In [ ]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

In [ ]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input1](x_0,EQ,2.967691214515491),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-1.408120229258977),
 docplex.mp.LinearConstraint[input6](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input7](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

In [ ]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

In [ ]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,2.967691214515491),
 docplex.mp.LinearConstraint[input GE](x_0,GE,2.967691214515491),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.408120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.790702170757714),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.24127975754059),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.24127975754059),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.6037614142464092),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [ ]:
def log_and_improve_explanation(minimal_explanation: list, epsilon: float):
	for constraint in minimal_explanation:
		testing_model.solve()
		print('Initial constraint:' + '\t', constraint)

		variable = constraint.lhs
		while testing_model.solution is None:
			if constraint.sense == docplex.mp.constants.ComparisonType.LE:
				if constraint.rhs.constant <= variable.ub:
					constraint.rhs += epsilon
				else:
					break
			elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
				if constraint.rhs.constant >= variable.lb:
					constraint.rhs -= epsilon
				else:
					break
			else:
				raise Exception('Constraint sense was neither LE nor GE')

			testing_model.solve()

		# Undo last operation
		if constraint.sense == docplex.mp.constants.ComparisonType.LE:
			constraint.rhs -= epsilon
		elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
			constraint.rhs += epsilon

		print('Final constraint:' + '\t', constraint)
		print()

In [ ]:
log_and_improve_explanation(linear_constraints, epsilon=0.01)

Initial constraint:	 input LE: x_0 <= 2.967691214515491
Final constraint:	 input LE: x_0 <= 5.127691214515445

Initial constraint:	 input GE: x_0 >= 2.967691214515491
Final constraint:	 input GE: x_0 >= 1.427691214515511

Initial constraint:	 input LE: x_3 <= -1.408120229258977
Final constraint:	 input LE: x_3 <= -1.408120229258977

Initial constraint:	 input GE: x_3 >= -1.408120229258977
Final constraint:	 input GE: x_3 >= -1.518120229258977

Initial constraint:	 input LE: x_5 <= -0.790702170757714
Final constraint:	 input LE: x_5 <= -0.790702170757714

Initial constraint:	 input GE: x_5 >= -0.790702170757714
Final constraint:	 input GE: x_5 >= -0.790702170757714

Initial constraint:	 input LE: x_6 <= 4.24127975754059
Final constraint:	 input LE: x_6 <= 4.24127975754059

Initial constraint:	 input GE: x_6 >= 4.24127975754059
Final constraint:	 input GE: x_6 >= 4.24127975754059

Initial constraint:	 input LE: x_7 <= -0.3615292659832898
Final constraint:	 input LE: x_7 <= -0.36152926598

In [ ]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.127691214515445),
 docplex.mp.LinearConstraint[input GE](x_0,GE,1.427691214515511),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.518120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.790702170757714),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.24127975754059),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.24127975754059),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.6037614142464092),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [ ]:
number_of_inputs = len(dataframe.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

In [ ]:
improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.127691214515445),
 docplex.mp.LinearConstraint[input GE](x_0,GE,1.427691214515511),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.518120229258977),
 docplex.mp.LinearConstraint[input](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input](x_8,EQ,-0.6037614142464092)]

### Pretty Printing

In [ ]:
def get_variable_index(variable: docplex.mp.dvar.Var) -> int:
	index = variable.name.split('_')[1]
	return int(index)

In [ ]:
def represent_constraint_with_feature_name(constraint: docplex.mp.constr.LinearConstraint):
	variable = constraint.lhs
	index = get_variable_index(variable)
	feature_name = dataframe.columns[index]
	return f'{feature_name} {constraint.sense.operator_symbol} {constraint.rhs}'

In [ ]:
def simple_print_explanation(explanation: list[docplex.mp.constr.LinearConstraint]):
	for e in explanation:
		print(represent_constraint_with_feature_name(e))

In [ ]:
def group_by_name(constraints: list[docplex.mp.constr.LinearConstraint]):
	group = {}
	for c in constraints:
		variable = c.lhs
		i = get_variable_index(variable)
		feature_name = dataframe.columns[i]
		if feature_name not in group:
			group[feature_name] = []
		group[feature_name].append(c)

	return group

In [ ]:
def get_interval_distance(constraints: list[docplex.mp.constr.LinearConstraint]):
	if len(constraints) == 1:
		return 0
	elif len(constraints) == 2:
		c_0 = constraints[0]
		c_1 = constraints[1]
		assert c_0.lhs.name == c_1.lhs.name
		distance = c_0.rhs.constant - c_1.rhs.constant
		return abs(distance)
	else:
		ValueError()

In [ ]:
def pretty_print_explanation(explanation: list[docplex.mp.constr.LinearConstraint]):
	group = group_by_name(improved_explanation)
	for (feature, constraints) in group.items():
		if len(constraints) == 2:
			c_0 = constraints[0]
			c_1 = constraints[1]
			le_operator = docplex.mp.constants.ComparisonType.LE
			if c_0.sense == le_operator:
				c_plus = c_0
				c_minus = c_1
			else:
				c_minus = c_0
				c_plus = c_1
			print(f'{c_minus.rhs} {le_operator.operator_symbol} {feature} {le_operator.operator_symbol} {c_plus.rhs}')
		else:
			print(represent_constraint_with_feature_name(constraints[0]))

In [ ]:
pretty_print_explanation(improved_explanation)

1.427691214515511 <= RI <= 5.127691214515445
-1.518120229258977 <= Al <= -1.408120229258977
K == -0.790702170757714
Ca == 4.24127975754059
Ba == -0.3615292659832898
Fe == -0.6037614142464092


In [ ]:
group = group_by_name(improved_explanation)
for feature in group:
	print(get_interval_distance(group[feature]))

3.699999999999934
0.1100000000000001
0
0
0
0


### Anchor

In [ ]:
from anchor import utils

In [ ]:
d = utils.load_csv_dataset(
	data=f'datasets/{dataset_name}/test.csv',
	target_idx=-1,
	feature_names=['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','target'],
	skip_first=True
)

In [ ]:
from anchor import anchor_tabular

In [ ]:
explainer = anchor_tabular.AnchorTabularExplainer(
	d.class_names,
	d.feature_names,
	d.train,
	d.categorical_names)

In [ ]:
predict_fn = lambda x: tf.argmax(keras_model.predict(x)[0]).numpy().reshape(1)

In [ ]:
for a in d.train:
	a == data[i]

/tmp/ipykernel_19251/1890216175.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  a == data[i]


In [ ]:
exp = explainer.explain_instance(data[i, :-1], predict_fn)

1/1 [==============================] - 0s 28ms/step


In [ ]:
for name in exp.names():
	print(name)

RI > -0.64
K > -0.60
Na <= 0.84
Si > -0.81
Al > -0.57


In [ ]:
pretty_print_explanation(improved_explanation)

1.427691214515511 <= RI <= 5.127691214515445
-1.518120229258977 <= Al <= -1.408120229258977
K == -0.790702170757714
Ca == 4.24127975754059
Ba == -0.3615292659832898
Fe == -0.6037614142464092


In [ ]:
simple_print_explanation(improved_explanation)

RI <= 5.127691214515445
RI >= 1.427691214515511
Al <= -1.408120229258977
Al >= -1.518120229258977
K == -0.790702170757714
Ca == 4.24127975754059
Ba == -0.3615292659832898
Fe == -0.6037614142464092


In [ ]:
print('Precision: %f' % exp.precision())
print('Coverage:  %f' % exp.coverage())

Precision: 0.252684
Coverage:  0.386700


## Australian